In [ ]:
# load data
df = pd.read_csv("Data_Marketing_Customer_Analysis_Round3.csv")
# create dataframes with numerical...
df_num = df[df.select_dtypes(include=[np.number]).columns].drop("total_claim_amount", axis=1)
# ...and categorical data only
df_cat = df[df.select_dtypes(include=[object]).columns].drop("effective_to_date", axis=1)

# 0. Data Exploration

In [ ]:
# use pairplot to find correclated features
sns.pairplot(df_num)

In [ ]:
# plot the most correlated features against each otherS
plt.scatter(x=df_num["monthly_premium_auto"], y=df_num["customer_lifetime_value"], c="r")

In [ ]:
# determine correlation matrix and plot correlation heatmap
corr = df_num.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True # trick to filter out the upper-right triangle, which is redundant due to symmetry
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(16, 14))
    ax = sns.heatmap(corr, mask=mask,cmap='coolwarm', vmin=-1,vmax=1,annot=True, square=True)

In [ ]:
#print(df_num.columns)
# kick out any highly correlated features
CORR_THRESH = 0.80
corr_matrix = df_num.corr().abs()
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(bool))
corrd_cols = [column for column in upper_triangle.columns if any(upper_triangle[column] > CORR_THRESH)]
df_num.drop(corrd_cols,axis=1,inplace=True)
#print(df_num.columns)

In [ ]:
# plot histplots of the numerical data
df_num.hist(figsize=(11,12))

In [ ]:
# monthly premium auto is scewed to the left
#sns.histplot(df_num["income"], kde=1)
sns.histplot(df_num["monthly_premium_auto"], kde=1)

In [ ]:
# one-hot encode the categorical features
cols_to_dummify = df_cat.columns
df_dums = pd.DataFrame()
for i, col in enumerate(cols_to_dummify):
    df_dummies = pd.get_dummies(df_cat[col], drop_first=1)
    # concatenate the original dataframe with the dummy variables
    df_dums = pd.concat([df_dums, df_dummies], axis=1)

# 1. X-y split

In [ ]:
from scipy.stats import zscore

In [ ]:
# filter the dataframe to remove the outliers
def remove_outliers(num_df):
    z = num_df.apply(zscore)
    threshold = 3
    num_df = num_df[(z < threshold).all(axis=1)]
    return num_df

In [ ]:
# load the data with dummies and ordinally encoded categorical data and then load the numerical data
# define numerical and categorical Xs and target feature y
categoricalX = pd.read_csv("dum_df.csv")
numericalX = pd.read_csv("num_df.csv").drop("total_claim_amount", axis=1)

X = pd.concat([categoricalX.iloc[remove_outliers(numericalX).index], remove_outliers(numericalX[numericalX.columns])], axis=1)
y = df.iloc[remove_outliers(numericalX).index]["total_claim_amount"]

In [ ]:
X.isna().any()
# no need to drop anything

# 2. Test - train split

In [ ]:
RAND_STATE = 34 # for reproducible shuffling
TT_RATIO = 0.3 # test/train

In [ ]:
# test-train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TT_RATIO, random_state=RAND_STATE)
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
y_train = pd.DataFrame(y_train)
X_train.head(10)

# 3. Standardize data

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# apply the StandardScaler to scale the distributions
ss = StandardScaler()
monthly_premium_transformed = ss.fit_transform(numericalX["monthly_premium_auto"].to_numpy().reshape(-1,1))

# original distribution
sns.displot(numericalX["monthly_premium_auto"], kde=1)
# normalized distribution done with StandardScaler
sns.displot(monthly_premium_transformed, kde=1)

In [ ]:
# apply the PowerTransformer to scale the distributions
pt = PowerTransformer()
monthly_premium_transformed = pt.fit_transform(df_num['monthly_premium_auto'].to_numpy().reshape(-1,1))

# original distribution
sns.displot(numericalX["monthly_premium_auto"], kde=1)
# normalized and normally transformed distribution done with PowerTransform
sns.displot(monthly_premium_transformed, kde=1)

In [ ]:
print("the parameters used to transform monthly_premium_transformed (pt) are", pt.get_params(), "\nlambdas:" , pt.lambdas_)
# parameter used in the power transformation

print("the parameters used to transform monthly_premium_transformed (ss) are", ss.get_params())

In [ ]:
ss = ColumnTransformer([("ss", ss, list(numericalX.columns))],
                        remainder='drop',verbose_feature_names_out=True,verbose=True).fit(X_train)
X_train_ss = pd.DataFrame(ss.fit_transform(X_train), columns=ss.get_feature_names_out())
X_test_ss = pd.DataFrame(ss.fit_transform(X_test), columns=ss.get_feature_names_out())

In [ ]:
# using the ColumnTransformer to transform the numerical columns with the PowerTransformer
ct = ColumnTransformer([("pt", pt, list(numericalX.columns))],
                        remainder='drop',verbose_feature_names_out=True,verbose=True).fit(X_train)
X_train_ct = pd.DataFrame(ct.transform(X_train),columns=ct.get_feature_names_out())
X_test_ct = pd.DataFrame(ct.transform(X_test),columns=ct.get_feature_names_out())

In [ ]:
X_test_ss

# 4. Apply linear regression

## 4.1 OLS

In [ ]:
X_train_const_ss = sm.add_constant(X_train_ss.to_numpy()) # adding a constant

model_ss = sm.OLS(y_train, X_train_const_ss).fit()
# predictions_train = model.predict(X_train_const_ct)

X_test_const_ss = sm.add_constant(X_test_ss) # adding a constant
predictions_test = model_ss.predict(X_test_const_ss)
print_model = model_ss.summary()
print(print_model)

In [ ]:
X_train_const_ct = sm.add_constant(X_train_ct.to_numpy()) # adding a constant

model_ct = sm.OLS(y_train, X_train_const_ct).fit()
# predictions_train = model.predict(X_train_const_ct)

X_test_const_ct = sm.add_constant(X_test_ct) # adding a constant
predictions_test = model_ct.predict(X_test_const_ct)
print_model = model_ct.summary()
print(print_model)

## 3.2 SciKit Learn

In [ ]:
# standard scaled train set
model_ss=LinearRegression()    # model
model_ss.fit(X_train_ss, y_train)   # model train

In [ ]:
# power transformed train set
model_ct=LinearRegression()    # model
model_ct.fit(X_train_ct, y_train)   # model train

# 5. Model interpretation

In [ ]:
print(model_ss.coef_, "\n", model_ct.coef_)

In [ ]:
print(model_ss.intercept_, model_ct.intercept_)

In [ ]:
# standard scaler data
y_pred_ss = pd.DataFrame(model_ss.predict(X_test_ct),columns = ['target_d'] )      # model prediction
y_pred_train_ss = pd.DataFrame(model_ss.predict(X_train_ct),columns = ['target_d'])

In [ ]:
# power transformer data
y_pred_ct = pd.DataFrame(model_ct.predict(X_test_ct),columns = ['target_d'] )      # model prediction
y_pred_train_ct = pd.DataFrame(model_ct.predict(X_train_ct),columns = ['target_d'])

In [ ]:
display(y_pred_ss, y_pred_ct)

In [ ]:
display(y_pred_train_ss, y_pred_train_ct)

# Model evaluation

In [ ]:
y_test = y_test.to_numpy().reshape(-1,1)

In [ ]:
# PLOTS FOR POWER TRANSFORMED DATA
# Make an scatter plot y_pred vs y
# What kind of plot you will get if all the predictions are ok?
# A stright line

fig, ax = plt.subplots(1,3,figsize=(14,4))
ax[0].plot(y_pred_ct, y_test, 'o')
ax[0].set_xlabel("y_test")
ax[0].set_ylabel("y_pred")
ax[0].set_title("Test Set - Predicted vs real")

# Get a histogram of the residuals ie: y - y_pred.  Homoscedasticity
# It resembles a normal distribution?
ax[1].hist(y_test - y_pred_ct)
ax[1].set_xlabel("Test y - y_pred")
ax[1].set_title("Test Set Residual histogram")

ax[2].plot(y_pred_ct,y_pred_ct.to_numpy() - y_test,"o")
ax[2].set_xlabel("predicted")
ax[2].set_ylabel("residuals")
ax[2].set_title("Residuals by Predicted")
ax[2].plot(y_pred_ct,np.zeros(len(y_pred_ct)),linestyle='dashed')

In [ ]:
# more fancy with seaborn
yp_ = y_pred_ct
yt_ = y_test
sns.regplot(x=yp_,y=yt_,scatter_kws={"color": "red"}, line_kws={"color": "black"})

## checking Mean squared error for power transformed data

In [ ]:
print(mse(y_test,y_pred_ct))
print(mae(y_test,y_pred_ct))
# prediction on the train set
print(mse(y_train,y_pred_train_ct))

## checking Mean squared error for standard scaler transformed data

In [ ]:
print(mse(y_test,y_pred_ss))
print(mae(y_test,y_pred_ss))
# prediction on the train set
print(mse(y_train,y_pred_train_ss))

## and now the root of mean squares error